In [1]:
from maraboupy import Marabou, MarabouCore
import numpy as np
import pandas as pd
import csv
import os
import sys

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def write_values_to_csv(values_dict, filename):
    # 获取当前脚本的目录
    current_directory = os.path.dirname(os.path.abspath(__file__))

    # 将文件名与当前目录连接起来
    full_path = os.path.join(current_directory, filename)

    # 打开文件进行追加
    with open(full_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # 提取字典中的值并写入一行
        values = list(values_dict.values())
        writer.writerow(values)

    print(f"Values written to {full_path}")


In [3]:
    # bounds = pd.read_csv(
    #     '/Code/Marabou/Behaviour_analysis_misclassification/bounds_3std.csv')
    # bounds = pd.read_csv(
    #     '/home/adam/Experiment/Code/Marabou/Behaviour_analysis_misclassification/bounds_fuzz90.csv'
    # )

In [4]:
# bounds = pd.read_csv(
#     '/home/adam/Experiment/Code/Marabou/Behaviour_analysis_misclassification/Bounds/HR-STS/bounds_mixed_inputspacetop90.csv'
#     )
# high_fatigue = pd.DataFrame(bounds)

file_name = 'model_without_softmax.onnx'
# network = Marabou.read_onnx(file_name, inputNames='dense_0_input', outputNames='dense_62', reindexOutputVars=True)
network = Marabou.read_onnx(file_name)

In [14]:
# 获取输入和输出变量
inputVars = network.inputVars[0][0]
outputVars = network.outputVars[0]

low_fatigue_idx = 0
medium_fatigue_idx = 1
high_fatigue_idx = 2
len(outputVars)

1

In [11]:
high_fatigue = (
    pd.read_csv('statistic_analysis/dataset_statistics_fatigue_level_2.csv'))
high_fatigue= high_fatigue['mean'][:-1]


In [16]:
# 定义特定的输入点
specific_input = high_fatigue  # 示例输入
for i in range(len(specific_input)):
    network.setLowerBound(inputVars[i], specific_input[i])
    network.setUpperBound(inputVars[i], specific_input[i])

# 定义输出条件
# 例如，验证模型是否预测特定类别
desired_output_class = 2  # 假设的类别索引
for i in range(len(outputVars)):
    if i != desired_output_class:
        network.addInequality([outputVars[0][i], outputVars[0][desired_output_class]], [1, -1], 0)

In [24]:

# 运行验证
vals = network.solve()[0] 

# 检查结果
if vals == "unsat":
    print("不存在满足条件的解")
else:
    print("存在满足条件的解")


unsat
不存在满足条件的解


'unsat'

In [ ]:

# 定义输入范围（如果有特定的输入范围）
# 例如，为每个输入变量设置范围
for var in inputVars:
    network.setLowerBound(var, lower_bound)  # 输入变量的下界
    network.setUpperBound(var, upper_bound)  # 输入变量的上界

# 定义输出约束
# 确保 'high fatigue' 的输出总是高于 'low' 和 'medium' fatigue
network.addInequality([outputVars[high_fatigue_idx], outputVars[low_fatigue_idx]], [1, -1], 0)   # high > low
network.addInequality([outputVars[high_fatigue_idx], outputVars[medium_fatigue_idx]], [1, -1], 0) # high > medium

# 解决验证问题
options = Marabou.createOptions(solveWithMILP=True)  # 使用MILP解决更复杂的问题
vals, stats = network.solve(options=options)

# 检查是否找到解 
if len(vals) > 0:
    print("找到满足条件的解")
else:
    print("未找到满足条件的解，可能意味着在指定条件下不存在违反约束的情况")

In [ ]:

    # High fatigue query
    inputVars = network.inputVars[0][0]

    # Set the upper bound of each input variable to the high_fatigue upper bound
    for i, (var_name, bounds) in enumerate(high_fatigue.iterrows()):
        network.setUpperBound(inputVars[i], bounds['upper_bound'])
        network.setLowerBound(inputVars[i], bounds['lower_bound'])

    # Set the lower and upper bounds for the output variable(0-6 means low or medium fatigue)
    outputVars = network.outputVars[0]
    for i in range(len(outputVars)):
        for j in range(len(outputVars[i])):
            network.setLowerBound(outputVars[i][j], 0)
            network.setUpperBound(outputVars[i][j], 6)

    options = Marabou.createOptions(numWorkers=20, initialTimeout=5, initialSplits=100, onlineSplits=100,
                                    timeoutInSeconds=1800, timeoutFactor=1.5,
                                    verbosity=2, snc=True, splittingStrategy='auto',
                                    sncSplittingStrategy='auto', restoreTreeStates=False,
                                    splitThreshold=20, solveWithMILP=False, dumpBounds=True)
    # Execute query in a loop
    try:

        while True:
            result = network.solve(
                filename='/home/adam/Experiment/Code/Marabou/logs/Behviour_analysis_mixedtop90.log',
                options=options)
            status, values, stats = result
            write_values_to_csv(values, 'HR-STSmixed_results/highfatigue_mixedtop90_005.csv')
            time += stats.getTotalTimeInMicro()
            count += 1

            # if time > 180000000:
            #     print("Timeout.")
            #     print("Time:", time)
            #     break

            # If return 'sat', which means there are some high fatigue data in the dataset is classified as low or medium fatigue
            if status == "sat":
                print("Solution: ", count, " found!")
                # print("Values:", values)

                print("Time:", time)

                # If result is SAT, add a new constraint to exclude the current solution
                for i in range(len(inputVars)):
                    # Create a constraint that the variable is less than the current solution
                    eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
                    eq1.addAddend(1, inputVars[i])
                    # Subtract a small value to avoid the current solution
                    eq1.setScalar(values[inputVars[i].item()] - 0.05)

                    # Create a constraint that the variable is greater than the current solution
                    eq2 = MarabouCore.Equation(MarabouCore.Equation.GE)
                    eq2.addAddend(1, inputVars[i])
                    # Add a small value to avoid the current solution
                    eq2.setScalar(values[inputVars[i].item()] + 0.05)

                    # Add the disjunction of the two constraints to the network
                    network.addDisjunctionConstraint([[eq1], [eq2]])

            elif status == "unsat":
                print("No solution found.")
                print("Time:", time)
                break

            pass

    except KeyboardInterrupt:
        print("Interrupted")
        sys.exit()


if __name__ == "__main__":
    main()
